In [ ]:
from pyHalo.pyhalo import pyHalo
import numpy as np
from time import time
import matplotlib.pyplot as plt

## To generate realizations with pyHalo you first have to specify a number of keyword arguments to specify mass definitions, lens/source redshift, etc.

In [ ]:
zlens, zsource = 0.5, 2.
rendering_volume = 'DOUBLE_CONE' # other option is CYLINDER
cone_opening_angle = 6. # the opening angle of the cone inside which halos are rendered, specified in arcseconds
# A reasonable choice would be opening_angle = 6 * R_ein, where R_ein is the Einstein radiuss of the lens in arcsec
kwargs_halo_mass_function = {'geometry_type': rendering_volume}


# properties of the realization
mass_definition = 'TNFW' # truncated NFW profile, other options are 
# 'NFW', 'SIDM_TNFW' (has a core), 'PT_MASS' (point mass), 'SIS', 'PJAFFE' (truncated SIS)

log_mlow = 6. # log(minimum halo mass)
log_mhigh = 10 # log(maximum halo mass)
power_law_index = -1.9 # logarithmic slope of the subhalo mass function (SHMF)
delta_power_law_index = 0. # change in logarithmic slope applied to both the SHMF and line of sight HMF
LOS_norm = 1. # rescales the line of sight halo mass function relative to Sheth-Tormen
sigma_sub = 0.02 # normalization of the SHMF (same parameterization as Gilman et al. 2020)
r_tidal = '0.25Rs' # subhalos are distributed in 3D following a cored NFW profile with a core radius r_tidal. 
# the scale radius is computed from the host halo mass you specify 
log_m_host = 13.

# since we add halos to Universe we have to subtract what we add so that on 
# average the density of the Universe is rho_crit. This is accomplished by adding sheets of negative convergence;
# use these keywords to specify in what mass range to compute the correction. Since we are adding halos everywhere between
# 10^6 and 10^10 we'll keep these the same as log_mlow and log_mhigh
log_mass_sheet_min = 6
log_mass_sheet_max = 10

# now we can create an instance of pyhalo, this assumes a default cosmology specified in pyhalo.defaults
pyhalo = pyHalo(zlens, zsource, kwargs_halo_mass_function=kwargs_halo_mass_function)

# This will assume a default mass-concentration relation using the Dimer and Joyce 2019 model
t0 = time()
realization_kwargs = {'mass_func_type': 'POWER_LAW', 'log_mlow': log_mlow, 'log_mhigh': log_mhigh, 
                      'log_mass_sheet_min': log_mlow, 'log_mass_sheet_max': log_mhigh, 
                      'mdef_main': 'TNFW','mdef_los': 'TNFW', 'sigma_sub': sigma_sub,
                      'delta_power_law_index': delta_power_law_index, 'cone_opening_angle': cone_opening_angle, 
                      'log_m_host': log_m_host, 'power_law_index': power_law_index, 'r_tidal': r_tidal,
                      'LOS_normalization': LOS_norm}

# for line of sight + subhalos
realization_type = 'composite_powerlaw'

# this creates a list of realizations of length nrealizations
realization = pyhalo.render(realization_type, realization_kwargs, nrealizations=1)[0]
tend = time()
Nhalos = len(realization.halos)
print('rendered '+str(Nhalos)+' halos in '+str(tend - t0)+' sec')

## pyhalo is designed to be used with the software lenstronomy (https://github.com/sibirrer/lenstronomy)
## In one step, you can go from a pyhalo realization to an instance of LensModel

In [ ]:
lens_model_list, lens_redshift_array, kwargs_halos, _ = realization.lensing_quantities()
# the last argument is only used for lens models with numerically-tabulated deflection angles
# note that len(lens_model_list) != len(realization.halos) because the lens_model_list includes 
# the negative convergence sheets 

from lenstronomy.LensModel.lens_model import LensModel

# You should use the same astropy class created in pyhalo to do lensing computations
astropy_instance = pyhalo.astropy_cosmo
lens_model_halos_only = LensModel(lens_model_list, zlens, zsource, lens_redshift_array, cosmo=astropy_instance, multi_plane=True)

_x = _y = np.linspace(-1.5, 1.5, 100)
xx, yy = np.meshgrid(_x, _y)
shape0 = xx.shape
kappa_subs = lens_model_halos_only.kappa(xx.ravel(), yy.ravel(), kwargs_halos).reshape(shape0)

fig = plt.figure(1)
fig.set_size_inches(8,8)
plt.imshow(kappa_subs, vmin=-0.1, vmax=0.1, cmap='bwr')

## Most of the time you'll combine the substructure realization with a macromodel

In [ ]:
# power law plus shear macromodel
kwargs_macromodel = [{'theta_E': 1., 'center_x': 0., 'center_y': 0., 'e1': 0.2, 'e2': -0.1, 'gamma': 2.},
                    {'gamma1': -0.04, 'gamma2': 0.09}]

lens_model_list_macro = ['EPL', 'SHEAR']
lens_model_list_full = lens_model_list_macro + lens_model_list
lens_redshift_list_full = [zlens, zlens] + list(lens_redshift_array)
kwargs_lens_full = kwargs_macromodel + kwargs_halos

lens_model_full = LensModel(lens_model_list_full, zlens, zsource, lens_redshift_list_full, cosmo=astropy_instance, multi_plane=True)
lens_model_macro = LensModel(lens_model_list_macro)

kappa_full = lens_model_full.kappa(xx.ravel(), yy.ravel(), kwargs_lens_full).reshape(shape0)
kappa_macro = lens_model_macro.kappa(xx.ravel(), yy.ravel(), kwargs_macromodel).reshape(shape0)

In [ ]:
fig = plt.figure(1)
fig.set_size_inches(8,8)
nonlinear_convegence = kappa_full - kappa_macro
plt.imshow(nonlinear_convegence, vmin=-0.1, vmax=0.1, cmap='bwr')